<a href="https://colab.research.google.com/github/jcmachicao/MachineLearningAvanzado_UC_2025/blob/main/U4__Monitor_Consistencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitor de Consistencia

In [1]:
import torch
import torch.nn as nn
from typing import Dict, List, Tuple
import numpy as np

In [2]:
class ConsistencyMonitor:
    def __init__(self, tolerance_threshold: float = 0.1):
        self.tolerance_threshold = tolerance_threshold
        self.historical_predictions: List[np.ndarray] = []
        self.consistency_scores: List[float] = []

    def check_temporal_consistency(self, new_prediction: np.ndarray) -> float:
        """Check if new predictions are consistent with historical trends."""
        if len(self.historical_predictions) < 2:
            self.historical_predictions.append(new_prediction)
            return 1.0

        # Calculate trend consistency
        prev_trend = self.historical_predictions[-1] - self.historical_predictions[-2]
        new_trend = new_prediction - self.historical_predictions[-1]

        # Normalize trends
        prev_trend_norm = prev_trend / (np.linalg.norm(prev_trend) + 1e-7)
        new_trend_norm = new_trend / (np.linalg.norm(new_trend) + 1e-7)

        # Calculate consistency score using cosine similarity
        consistency_score = np.dot(prev_trend_norm.flatten(), new_trend_norm.flatten())

        # Update history
        self.historical_predictions.append(new_prediction)
        self.consistency_scores.append(consistency_score)

        return consistency_score

    def detect_anomalies(self) -> List[int]:
        """Detect timepoints where consistency was violated."""
        anomalies = []
        for i, score in enumerate(self.consistency_scores):
            if score < self.tolerance_threshold:
                anomalies.append(i)
        return anomalies

    def get_recovery_suggestion(self, anomaly_index: int) -> str:
        """Suggest recovery strategy based on anomaly pattern."""
        if anomaly_index >= len(self.consistency_scores):
            return "Invalid anomaly index"

        score = self.consistency_scores[anomaly_index]
        if score < 0:
            return "Critical: Prediction trend reversed. Consider model retraining."
        elif score < self.tolerance_threshold/2:
            return "Warning: Significant consistency drop. Check input data quality."
        else:
            return "Minor: Slight consistency deviation. Monitor next predictions."

In [3]:
class ConsistentModel(nn.Module):
    def __init__(self, base_model: nn.Module):
        super().__init__()
        self.base_model = base_model
        self.consistency_monitor = ConsistencyMonitor()

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, float]:
        # Get base model prediction
        prediction = self.base_model(x)

        # Check consistency
        consistency_score = self.consistency_monitor.check_temporal_consistency(
            prediction.detach().numpy()
        )

        # If consistency is low, trigger recovery mechanism
        if consistency_score < self.consistency_monitor.tolerance_threshold:
            # Example recovery: Smooth prediction with historical average
            historical_tensor = torch.tensor(self.consistency_monitor.historical_predictions[-3:])
            prediction = (prediction + historical_tensor.mean(dim=0)) / 2

        return prediction, consistency_score

In [4]:
# Create base model
base_model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)

In [5]:
# Wrap with consistency monitoring
consistent_model = ConsistentModel(base_model)

In [6]:
# Simulate predictions
for _ in range(10):
    x = torch.randn(1, 10)
    prediction, consistency = consistent_model(x)
    print(f"Prediction shape: {prediction.shape}, Consistency: {consistency:.4f}")

    # Check for anomalies
    anomalies = consistent_model.consistency_monitor.detect_anomalies()
    if anomalies:
        print("Detected anomalies at timesteps:", anomalies)
        for anomaly_idx in anomalies:
            suggestion = consistent_model.consistency_monitor.get_recovery_suggestion(anomaly_idx)
            print(f"Recovery suggestion: {suggestion}")

Prediction shape: torch.Size([1, 2]), Consistency: 1.0000
Prediction shape: torch.Size([1, 2]), Consistency: 1.0000
Prediction shape: torch.Size([1, 2]), Consistency: -0.8868
Detected anomalies at timesteps: [0]
Recovery suggestion: Critical: Prediction trend reversed. Consider model retraining.
Prediction shape: torch.Size([1, 2]), Consistency: -0.8216
Detected anomalies at timesteps: [0, 1]
Recovery suggestion: Critical: Prediction trend reversed. Consider model retraining.
Recovery suggestion: Critical: Prediction trend reversed. Consider model retraining.
Prediction shape: torch.Size([1, 2]), Consistency: 0.6311
Detected anomalies at timesteps: [0, 1]
Recovery suggestion: Critical: Prediction trend reversed. Consider model retraining.
Recovery suggestion: Critical: Prediction trend reversed. Consider model retraining.
Prediction shape: torch.Size([1, 2]), Consistency: -0.6192
Detected anomalies at timesteps: [0, 1, 3]
Recovery suggestion: Critical: Prediction trend reversed. Consid

<ipython-input-3-4a623b60519d>:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  historical_tensor = torch.tensor(self.consistency_monitor.historical_predictions[-3:])


# Monitor Simple de Consistencia

In [7]:
def analizar_consistencia(predicciones):
    """
    Analiza la consistencia de una serie de predicciones usando
    tanto la dirección como la magnitud del cambio.
    """
    if len(predicciones) < 3:
        return {"consistencia": 1.0, "estado": "OK"}

    # Calculamos tendencias
    tendencia_previa = predicciones[-2] - predicciones[-3]
    tendencia_actual = predicciones[-1] - predicciones[-2]

    # Calculamos score de consistencia usando varios factores

    # 1. Dirección del cambio (usando producto punto normalizado)
    norm_previa = tendencia_previa / (abs(tendencia_previa) + 1e-7)
    norm_actual = tendencia_actual / (abs(tendencia_actual) + 1e-7)
    direccion_score = norm_previa * norm_actual

    # 2. Magnitud del cambio
    magnitud_previa = abs(tendencia_previa)
    magnitud_actual = abs(tendencia_actual)
    ratio_magnitud = min(magnitud_previa, magnitud_actual) / (max(magnitud_previa, magnitud_actual) + 1e-7)

    # Score combinado
    score = direccion_score * ratio_magnitud

    # Determinamos estado
    if direccion_score < 0:
        estado = f"CRÍTICO: Cambio de dirección (score={score:.2f})"
    elif score < 0.5:
        estado = f"ADVERTENCIA: Desviación significativa (score={score:.2f})"
    else:
        estado = f"OK (score={score:.2f})"

    return {
        "consistencia": score,
        "estado": estado,
        "tendencia_previa": tendencia_previa,
        "tendencia_actual": tendencia_actual,
        "direccion_score": direccion_score,
        "ratio_magnitud": ratio_magnitud
    }

In [8]:
# Ejemplos de uso
casos_prueba = [
    [1, 2, 3, 4, 5], # Caso 1: Consistente creciente
    [1, 2, 3, 2, 1], # Caso 2: Inconsistencia por cambio de dirección
    [1, 2, 3, 3.8, 3.9], # Caso 3: Desviación significativa
    [1, 3, 5, 3.8, 2.5] # Caso 3: Desviación significativa

]

In [9]:
for i, caso in enumerate(casos_prueba, 1):
    print(f"\nCaso {i}: {caso}")
    resultado = analizar_consistencia(caso)
    print(f"Score de consistencia: {resultado['consistencia']:.2f}")
    print(f"Estado: {resultado['estado']}")
    print(f"Tendencia previa: {resultado['tendencia_previa']:.2f}")
    print(f"Tendencia actual: {resultado['tendencia_actual']:.2f}")


Caso 1: [1, 2, 3, 4, 5]
Score de consistencia: 1.00
Estado: OK (score=1.00)
Tendencia previa: 1.00
Tendencia actual: 1.00

Caso 2: [1, 2, 3, 2, 1]
Score de consistencia: 1.00
Estado: OK (score=1.00)
Tendencia previa: -1.00
Tendencia actual: -1.00

Caso 3: [1, 2, 3, 3.8, 3.9]
Score de consistencia: 0.12
Estado: ADVERTENCIA: Desviación significativa (score=0.12)
Tendencia previa: 0.80
Tendencia actual: 0.10

Caso 4: [1, 3, 5, 3.8, 2.5]
Score de consistencia: 0.92
Estado: OK (score=0.92)
Tendencia previa: -1.20
Tendencia actual: -1.30
